# Tracking Edge Effect Study

## For PVSC Paper



In [3]:
import sys
import csv

sys.path.insert(0, '../bifacial_radiance')

try:
    from bifacial_radiance import *
    print "Imported!"
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')

Imported!


In [ ]:
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
writefiletitle = "C:\Users\silvana\Documents\RadianceScene\Output_Edge_Effect_Study_TRACKING_PVSC_Journal.csv"


# tracker geometry options:
module_height = 1.980  # module portrait dimension in meters
gcr = 0.35   # ground cover ratio,  = module_height / pitch
albedo = 0.62     # ground albedo
hub_height = 1.485   # tracker height at 0 tilt in meters (hub height)
limit_angle = 45 # tracker rotation limit angle

# LOOP VARIABLES FOR EDGE EFFECT STUDY
height_all = [1.485, 0.9]
nRows_all = [1,3,5,7,9]
nMods_all = [2,4,6,8,10,12,14,16,18,20]

## Begin 1-axis SAT specific functions
# create separate metdata files for each 1-axis tracker angle (5 degree resolution).  
# create cumulativesky functions for each tracker angle: demo.genCumSky1axis


with open (writefiletitle,'wb') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # Write Simulation Parameters (from setup file)

    outputheader=['Clearance', 'Rows', 'Modules', 'Min Back Irr', 'Mean Back Irr']
    sw.writerow(outputheader)


    for zz in range (0, len(height_all)):
        height_C = height_all[zz]

        for ii in range (0, len(nRows_all)):
            nRows = nRows_all[ii]

            for jj in range (1, len(nMods_all)):

                outputvalues = []
                nMods = nMods_all[jj]

                print "\n********************"
                print "--> Working on ", nMods, "modules, and ", nRows, " rows, ", height_C, " clearance"
                print "********************"

                trackerdict=[]
                demo2 = []

                demo2 = RadianceObj(path = testfolder)  # Create a RadianceObj 'object' named 'demo'
                demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
                epwfile = demo2.getEPW(42.22, -121.74) # KLAMATH Pull TMY weather data for any global lat/lon.  In this case, Richmond, VA
                metdata = demo2.readEPW(epwfile) # read in the weather data


                trackerdict = demo2.set1axis(metdata, limit_angle = limit_angle, backtrack = True, gcr = gcr)

                trackerdict = demo2.genCumSky1axis(trackerdict)
                # Create a new moduletype: Prism Solar Bi60. width = .984m height = 1.695m. 
                #demo2.makeModule(name='Prism Solar Bi60',x=0.984,y=module_height)
                # Create a new moduletype: Silfab 285 SLA-X. width = 990 mm, height = 1650 mm. 2-up landscape, so moduleheight = 2*990mm =1.980 m and x = 1.650,
                demo2.makeModule(name='Silfab 285 SLA-X',x=1.650,y=1.980)
                # print available module types
                demo2.printModules()

                # create a 1-axis scene using panels in portrait, 2m hub height, 0.33 GCR. NOTE: clearance is calculated at each step. hub height is constant
                sceneDict = {'pitch': height_C / gcr,'height':height_C,'orientation':'portrait'}  
                module_type = 'Silfab 285 SLA-X'
                trackerdict = demo2.makeScene1axis(trackerdict,module_type,sceneDict, nMods = nMods, nRows = nRows) #makeScene creates a .rad file with 20 modules per row, 7 rows.
                # TODO:  can this 20x7 scene be reduced in size without encountering edge effects?
                trackerdict = demo2.makeOct1axis(trackerdict)
                # Now we need to run analysis and combine the results into an annual total.  This can be done by calling scene.frontscan and scene.backscan
                trackerdict = demo2.analysis1axis(trackerdict)

                # the frontscan and backscan include a linescan along a chord of the module, both on the front and back.  
                # Return the minimum of the irradiance ratio, and the average of the irradiance ratio along a chord of the module.
                print('Annual RADIANCE bifacial ratio for 1-axis tracking: %0.3f - %0.3f' %(min(demo2.backRatio), np.mean(demo2.backRatio)) )

                outputvalues.append(height_C)
                outputvalues.append(nRows)
                outputvalues.append(nMods)
                outputvalues.append(min(demo2.backRatio))
                outputvalues.append(np.mean(demo2.backRatio))                
                sw.writerow(outputvalues)
                print(' ' )




********************
--> Working on  4 modules, and  1  rows,  1.485  clearance
********************
path = C:\Users\silvana\Documents\RadianceScene\Test
Getting weather file: USA_OR_Klamath.Falls.Intl.AP.725895_TMY3.epw  ... OK!


C:\Users\Silvana\Anaconda2\lib\site-packages\pvlib\tracking.py:423: RuntimeWarning: invalid value encountered in minimum
  temp = np.minimum(axes_distance*cosd(wid), 1)


Saving file EPWs\1axis_0.0.csv, # points: 108
Saving file EPWs\1axis_-30.0.csv, # points: 132
Saving file EPWs\1axis_35.0.csv, # points: 184
Saving file EPWs\1axis_5.0.csv, # points: 339
Saving file EPWs\1axis_-25.0.csv, # points: 214
Saving file EPWs\1axis_40.0.csv, # points: 135
Saving file EPWs\1axis_10.0.csv, # points: 112
Saving file EPWs\1axis_-20.0.csv, # points: 163
Saving file EPWs\1axis_45.0.csv, # points: 804
Saving file EPWs\1axis_15.0.csv, # points: 116
Saving file EPWs\1axis_-15.0.csv, # points: 121
Saving file EPWs\1axis_-45.0.csv, # points: 808
Saving file EPWs\1axis_20.0.csv, # points: 198
Saving file EPWs\1axis_-10.0.csv, # points: 304
Saving file EPWs\1axis_-40.0.csv, # points: 149
Saving file EPWs\1axis_25.0.csv, # points: 213
Saving file EPWs\1axis_-5.0.csv, # points: 68
Saving file EPWs\1axis_-35.0.csv, # points: 134
Saving file EPWs\1axis_30.0.csv, # points: 92
message: There were 68 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "C:\Users\Silvana\Anaconda2\lib\site-packages\zmq\eventloop\zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Silvana\Anaconda2\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\Silvana\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "C:\Users\Silvana\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "C:\Users\Silvana\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "C:\Users\Silvana\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "C:\Users\Silvana\Anaconda2\lib\site-packages\ipykernel\kernelba